In [1]:
pip show python-dotenv

Name: python-dotenv
Version: 1.0.1
Summary: Read key-value pairs from a .env file and set them as environment variables
Home-page: https://github.com/theskumar/python-dotenv
Author: Saurabh Kumar
Author-email: me+github@saurabh-kumar.com
License: BSD-3-Clause
Location: c:\Users\Yahia\Desktop\langchain_bot\.venv\Lib\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

In [3]:
with open("files/churchill_speech.txt") as f: #replace your textual data file here
    churchill_speech = f.read()
chunks = text_splitter.create_documents([churchill_speech] )

In [4]:
# print(chunks[3])
print(f'you have {len(chunks)} chunk')

you have 301 chunk


In [5]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model("text-embedding-3-small")
    total_tokens = sum( [len(enc.encode( page.page_content)) for page in texts] )
    print(f"Total tokens: {total_tokens}")
    print(f'total cost in USD: {total_tokens/1000*0.0004:.6f}')


print_embedding_cost(chunks)

Total tokens: 4817
total cost in USD: 0.001927


In [6]:
from pprint import pprint
import google.generativeai as genai
for model in genai.list_models():
    print(model.name)

c:\Users\Yahia\Desktop\langchain_bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision
models/embedding-001
models/text-embedding-004
models/aqa


In [54]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embedding.embed_query(chunks[0].page_content)
vector

[0.037844345,
 -0.00085658877,
 -0.060029667,
 -0.032104097,
 0.08814883,
 0.030916587,
 0.037259985,
 -0.025780361,
 0.007818491,
 0.07457992,
 -0.044059794,
 0.027255997,
 -0.013571122,
 0.024505947,
 0.004102403,
 -0.038960103,
 0.034522288,
 0.050970513,
 0.033086702,
 -0.03418318,
 0.013469413,
 0.024585757,
 0.004427917,
 0.0049011842,
 -0.0039580828,
 -0.030039325,
 0.00460884,
 -0.0513265,
 -0.050702784,
 0.020361904,
 -0.05639095,
 -0.0029218437,
 -0.046612952,
 0.015292161,
 0.0023734418,
 -0.04497213,
 0.0019555995,
 0.03230923,
 0.0071131736,
 0.023425318,
 0.0019835834,
 0.0049692173,
 -0.008327497,
 0.021372508,
 -0.000987992,
 0.00585285,
 -0.05374014,
 0.03889782,
 0.0080673145,
 -0.025570275,
 0.049641863,
 0.015060471,
 0.07188605,
 -0.058732994,
 0.00055982755,
 -0.05486938,
 0.056019716,
 -0.023651911,
 -0.076583296,
 0.01763208,
 0.0006022621,
 -0.010807971,
 0.015538362,
 0.026606014,
 -0.032902543,
 -0.07849998,
 -0.01065064,
 -0.007149997,
 0.036612008,
 0.01831

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
vector = embedding.embed_query(chunks[0].page_content)
vector

In [27]:
pip install -q pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip show pinecone-client

Name: pinecone-client
Version: 3.2.2
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: c:\Users\Yahia\Desktop\langchain_bot\.venv\Lib\site-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [55]:
import pinecone
from langchain_community.vectorstores import Pinecone
pc = pinecone.Pinecone()

In [65]:
for i in pc.list_indexes().names():
    pc.delete_index(i)
    print(f'destroying index {i}')
print("Done")

destroying index churchill-speech
Done


In [66]:
index_name = "churchill-speech"
if index_name not in pc.list_indexes().names():
    print(f"creating the index {index_name}")
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        # metric="euclidean",
        spec=pinecone.PodSpec(environment="gcp-starter"),
    )
    print(f'Done Creation of {index_name}')

creating the index churchill-speech
Done Creation of churchill-speech


In [67]:
vector_store = Pinecone.from_documents(chunks, embedding,index_name=index_name)

### loading the vector storage from an existing index

In [68]:
vector_store = Pinecone.from_existing_index(index_name, embedding)

## Asking Questions (Similarity Search)

### Using SearchSimilarity in VectorStorage

In [69]:
query = input("give your query to be searched:")
results=vector_store.similarity_search(query, 9)
for result in results:
    print(result.page_content)

French Armies who had entered Belgium at the appeal of the Belgian King; but this strategic fact
fall upon us. The King of the Belgians had called upon us to come to his aid. Had not this Ruler
involved almost certainly the destruction of the fine Belgian Army of over 20 divisions and the
the last moment, when Belgium was already invaded, King Leopold called upon us to come to his aid,
Belgium to keep on holding the right hand of the Belgians and to give their own right hand to a
and Belgium is a colossal military disaster. The French Army has been weakened, the Belgian Army
and British, out of the
held by the French troops.
the French and


### Using LLM

In [71]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.5)
retreiver = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":10})
chain = RetrievalQA.from_chain_type(
    retriever=retreiver,
    llm=llm,
    chain_type="stuff"
)

In [72]:
query = input("give your query to be searched:")
answer = chain.run(query)
print(answer)

Philippe
